## Part1:Training and Implementation of Decision Tree only on categorical data.

In [26]:
import numpy as np
import pandas as pd
import pprint
import matplotlib.pyplot as plt

#### Read the dataSet from csv file using pandas library

In [27]:
df=pd.read_csv("decision_Tree/train.csv")

In [28]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.10,0.90,7,286,4,0,1,0,sales,low
1,0.89,0.93,4,249,3,0,0,0,sales,low
2,0.38,0.50,2,132,3,0,1,0,accounting,low
3,0.95,0.71,4,151,4,0,0,0,sales,medium
4,0.84,0.84,5,163,3,0,0,0,technical,low


##### Drop the non-categorical columns

In [29]:
df=df.drop(['satisfaction_level', 'last_evaluation','number_project','average_montly_hours','time_spend_company'], axis=1)

#### See the type of objects and  to check missing values

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11238 entries, 0 to 11237
Data columns (total 5 columns):
Work_accident            11238 non-null int64
left                     11238 non-null int64
promotion_last_5years    11238 non-null int64
sales                    11238 non-null object
salary                   11238 non-null object
dtypes: int64(3), object(2)
memory usage: 439.1+ KB


In [31]:
df.head()

,Work_accident,left,promotion_last_5years,sales,salary
0,0,1,0,sales,low
1,0,0,0,sales,low
2,0,1,0,accounting,low
3,0,0,0,sales,medium
4,0,0,0,technical,low


In [32]:
df.shape

(11238, 5)

#### We split the dataset into 2 sets i.e trainingSet which is 80% of total data and testSet which is 20% of the total data.
1. This is done to avoid the overfitting of our classifier on datasets.
2. Our main objective is to classify the unseen examples better. So we train our classifier first on 80% data and then test with rest 20% data ,do check how well our classifier can work. 

In [33]:
split = int(0.8 * df.shape[0])

training_data = df[:split] # 80% of the total data
testing_data = df[split:]  # 20% of the total data

In [34]:
training_data.head()

,Work_accident,left,promotion_last_5years,sales,salary
0,0,1,0,sales,low
1,0,0,0,sales,low
2,0,1,0,accounting,low
3,0,0,0,sales,medium
4,0,0,0,technical,low


Entropy = -${P(yes)\log_{2}{P(yes)}}$ -P(no)${\log_{2}{P(no)}}$

In [35]:
def count_Unique(col):
    d={} # empty dictionary
    for i in col:
        if i not in d:
            d[i]=1
        else:
            d[i]+=1
    return d

def TotalEntropy(Z):
    freq=count_Unique(Z)
    ent=0.0
    total=len(Z)
    for i in freq:
        prob=freq[i]/total
        ent+=-prob*np.log2(prob)
    return ent
    

InformationGain = E($S_i$)- ${\sum_{i=0}^{N}{\frac{\left| S_i \right|}{\left| S \right|}   } }$${E(eachFeature)}$

In [36]:
def entropy_attribute(df,attribute):
    
    var=list(set(df[attribute]))
    entropy_final=0
    for i in var:
        val1=len(df[attribute][df[attribute]==i][df.left == 1])
        val2=len(df[attribute][df[attribute]==i])
        if val1 == 0 or val2 == 0 or val1==val2:
            entropy_temp = 0
        else:
            ent = val1 / (val2)
            entropy_temp=-((ent * (np.log2(ent))) + ((1 - ent) * (np.log2(1-ent))))
      
   
        weighted=val2/len(df)
        entropy_final+=weighted*entropy_temp

    return entropy_final

#### It calculates the information gain and returns the feature with max(info_gain)

In [37]:
def information_gain(df,attribute):
    if not attribute:
        return 'empty'
    
    totalEntropy=TotalEntropy(df['left'])
    max = 0
    select_attr = 0

    for i in attribute:
        wightedEntropy = entropy_attribute(df,i)
        info_gain = totalEntropy - wightedEntropy
        if info_gain >= max :
                 max = info_gain
                 select_attr = i

    return select_attr

### Build Decision Tree Function.

1. It will take dataframe and list of features as an argument
2. Then it will call information_gain function which return max info_gain feature.
3. Then it remove that feature from the list and make a node of it.
4. then we recursively call the make_tree function untill  list becomes empty or Entropy of particular node is 0.
5. If leaf node is not pure , then we no. of 1's and 0's then give answer accordingly


In [38]:
def make_tree(df, attribute, tree=None):
    
    node = information_gain(df,attribute)
    temp = []# empty list to hold updated features which are left after selecting node
    if node is 'empty':
        a1 = df['left'][df['left'] == 1].count()
        a2 = df['left'][df['left'] == 0].count()
        
        if a1 > a2:
            return 1
        else:
            return 0
        
        
    for i in attribute:
        temp.append(i) 
    temp.remove(node)
    
    
    collval = np.unique(df[node])
   
    
    if tree is None:
        tree = {}
        tree[node] = {}
        
    for value in collval:
        sub_tree = df[['salary','left','promotion_last_5years', 'sales','Work_accident']][df[node] == value]
        
        count = len(df[node][df[node] == value])
        val,counts = np.unique(sub_tree['left'],return_counts=True)
        

        if len(counts)==1:
            tree[node][value]= val[0]
            
        else:
            tree[node][value] = make_tree(sub_tree, temp)
            
    return tree


In [39]:
featureSets = ['sales','promotion_last_5years','salary',  'Work_accident']#only took feature of categorical values
tree = make_tree(training_data, featureSets)

pprint.pprint(tree)

{'Work_accident': {0: {'salary': {'high': {'sales': {'IT': {'promotion_last_5years': {0: 0}},
                                                     'RandD': {'promotion_last_5years': {0: 0}},
                                                     'accounting': {'promotion_last_5years': {0: 0,
                                                                                              1: 0}},
                                                     'hr': {'promotion_last_5years': {0: 0,
                                                                                      1: 0}},
                                                     'management': {'promotion_last_5years': {0: 0,
                                                                                              1: 0}},
                                                     'marketing': {'promotion_last_5years': {0: 0}},
                                                     'product_mng': {'promotion_last_5years': {0: 0}},
               

### Prediction Function 

In [40]:
def predict(inst,tree):
    for nodes in tree.keys():        
        value = inst[nodes]
        
        try:
            tree = tree[nodes][value]
            prediction = 0
        except:
            return 1
            
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break;                            
        
    return prediction

In [41]:
inst = training_data.iloc[6]
inst

Work_accident                     0
left                              0
promotion_last_5years             0
sales                    management
salary                          low
Name: 6, dtype: object

#### Testing on 1 example

In [42]:
data = {'Work_accident':0, 'sales':'management','salary':'medium','promotion_last_5years':0}
inst = pd.Series(data)
inst

Work_accident                     0
sales                    management
salary                       medium
promotion_last_5years             0
dtype: object

In [43]:
prediction = predict(inst, tree)
prediction

0

#### Testing on Test dataSet

In [44]:
prediction=[]
for i in range(0,len(testing_data)):
    inst = testing_data.iloc[i]
    pre = predict(inst, tree)
    prediction.append(pre)

### Accuracy, Precision,Recall and F1Score to measure effectiveness of our Classifier 

In [45]:
actual_result=testing_data['left'].tolist()
Tp=0
Tn=0
Fp=0
Fn=0
for i in range(0,len(actual_result)):
    if actual_result[i]==prediction[i] and actual_result[i]==1:
        Tp+=1
    elif actual_result[i]==prediction[i] and actual_result[i]==0:
        Tn+=1
    elif actual_result[i]==0 and prediction[i]==1:
        Fp+=1
    else:
        Fn+=1

accuracy=(Tp+Tn)/len(actual_result)*100
precision=Tp/(Tp+Fp)
recall=Tp/(Tp+Fn)
F1_Score=2*precision*recall/(recall+precision)



In [46]:
print(accuracy,"%")
print(precision)
print(recall)
print(F1_Score)

75.75622775800713 %
0.5
0.003669724770642202
0.007285974499089253
